In [1]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
from openbabel import openbabel
import pubchempy as pcp
import numpy as np
import py3Dmol
import os
import ipywidgets as widgets
from ipywidgets import interact, fixed, IntSlider, Text, Dropdown, ToggleButton, Button, FloatSlider, Checkbox
from IPython.display import display
obMol = openbabel.OBMol()
obConv = openbabel.OBConversion()
"""IMPORTANT: DO NOT USE ANY OTHER VARIABLES NAMED obMol OR obConv!!!"""

from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdCIPLabeler
from rdkit.Chem import rdAbbreviations
IPythonConsole.drawOptions.addAtomIndices = False
IPythonConsole.ipython_useSVG=True
IPythonConsole.molSize = 300,300

In [8]:
cache = DataCache()

In [7]:
class converter:
    def __init__(self, data: str, data_type: str, data_cache: DataCache):
        """Initializes the converter with the input data and its type"""
        self.data = data
        self.data_type = data_type.lower()
        self.data_cache = data_cache

    def convert(self, target_format: str):
        """Converts the input data to the target format"""
        target_format = target_format.lower()
        if self.data_type == 'name':
            self.data_cache.add_data('Name', self.data)
            smiles = self.name_to_smiles()
            sdf = self.smiles_to_sdf(smiles)
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.data
        elif self.data_type == 'smiles':
            self.data_cache.add_data('SMILES', self.data)
            smiles = self.data
            sdf = self.smiles_to_sdf(smiles)
            if target_format == 'smiles':
                return self.data
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name1()
        elif self.data_type == 'inchi':
            self.data_cache.add_data('InChi', self.data)
            smiles = self.inchi_to_smiles()
            sdf = self.smiles_to_sdf(smiles)
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.data
            elif target_format == 'inchikey':
                return self.smiles_to_inchikey(smiles)
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)
        elif self.data_type == 'inchikey':
            self.data_cache.add_data('InChiKey', self.data)
            smiles = self.inchikey_to_smiles()
            sdf = self.smiles_to_sdf(smiles)
            if target_format == 'smiles':
                return smiles
            elif target_format == 'cid':
                return self.smiles_to_cid(smiles)
            elif target_format == 'sdf':
                return sdf
            elif target_format == 'inchi':
                return self.smiles_to_inchi(smiles)
            elif target_format == 'inchikey':
                return self.data
            elif target_format == 'xyz':
                return self.sdf_to_xyz(sdf, smiles)
            elif target_format == 'zmat':
                return self.sdf_to_zmat(sdf, smiles)
            elif target_format == 'name':
                return self.smiles_to_name2(smiles)

    def name_to_smiles(self):
        """Converts molecule name to SMILES"""
        try:
            c = pcp.get_compounds(self.data, 'name')
            smiles = c[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def smiles_to_name1(self):
        """Converts SMILES to molecule name"""
        try:
            smi = pcp.get_compounds(self.data, 'smiles')
            name = smi[0].iupac_name
            self.data_cache.add_data('Name', name)
            return name
        except IndexError:
            return None

    def smiles_to_name2(self, smiles):
        """Converts SMILES to molecule name"""
        try:
            smi = pcp.get_compounds(smiles, 'smiles')
            name = smi[0].iupac_name
            self.data_cache.add_data('Name', name)
            return name
        except IndexError:
            return None

    def inchi_to_smiles(self):
        """Converts InChi to SMILES"""
        try:
            ic = pcp.get_compounds(self.data, 'inchi')
            smiles = ic[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def inchikey_to_smiles(self):
        """Converts InChiKey to SMILES"""
        try:
            ick = pcp.get_compounds(self.data, 'inchikey')
            smiles = ick[0].isomeric_smiles
            self.data_cache.add_data('SMILES', smiles)
            return smiles
        except IndexError:
            return None

    def smiles_to_cid(self, smiles):
        """Converts SMILES to CID"""
        try:
            c = pcp.get_cids(smiles, 'smiles', list_return='flat')
            cid = c[0]
            self.data_cache.add_data('CID', cid)
            return cid
        except IndexError:
            return None
        
    def smiles_to_sdf(self, smiles):
        """Converts SMILES to SDF"""
        try:
            cid = self.smiles_to_cid(smiles)
            name_file = self.smiles_to_name2(smiles)
            if cid:
                file_path = f'./3Dfiles/{name_file}'
                try:
                    pcp.download('SDF', f'{file_path}.sdf', cid, 'cid', overwrite=True)
                    return pcp.get_sdf(cid)
                except Exception as e:
                    print("Error during download:", e)
                    return None
            else:
                return None
        except IndexError:
            return None

    def smiles_to_inchi(self, smiles):
        """Converts SMILES to InChI"""
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("smiles", "inchi")
            obConv.ReadString(obMol, smiles)
            ic = obConv.WriteString(obMol)
            self.data_cache.add_data('InChi', ic)
            return ic
        except IndexError:
            return None

    def smiles_to_inchikey(self, smiles):
        """Converts SMILES to InChiKey"""
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("smiles", "inchikey")
            obConv.ReadString(obMol, smiles)
            ick = obConv.WriteString(obMol)
            self.data_cache.add_data('InChiKey', ick)
            return ick
        except IndexError:
            return None

    def sdf_to_xyz(self, sdf, smiles):
        """Converts SDF to XYZ"""
        name = self.smiles_to_name2(smiles)
        directory = './3Dfiles/'
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("sdf", "xyz")
            obConv.ReadString(obMol, sdf)
            xyz = obConv.WriteString(obMol)
            if not os.path.exists(directory):
                os.makedirs(directory)
            file_path = os.path.join('./3Dfiles/', f"{name}.xyz")
            with open(file_path, "w") as file:
                file.write(xyz)
            return xyz
        except IndexError:
            return None

    def sdf_to_zmat(self, sdf, smiles):
        """Converts SDF to zmat"""
        name = self.smiles_to_name2(smiles)
        directory = './3Dfiles/'
        try:
            obMol = openbabel.OBMol()
            obConv = openbabel.OBConversion()
            obConv.SetInAndOutFormats("sdf", "gzmat")
            obConv.ReadString(obMol, sdf)
            zmat = obConv.WriteString(obMol)
            if not os.path.exists(directory):
                os.makedirs(directory)
            file_path = os.path.join('./3Dfiles/', f"{name}.txt")
            with open(file_path, "w") as file:
                file.write(zmat)
            return zmat
        except IndexError:
            return None

In [2]:
class DataCache:
    def __init__(self):
        self.cache = {}

    def add_data(self, key, value):
        if key not in self.cache:
            self.cache[key] = [value]
        else:
            if value not in self.cache[key]:
                self.cache[key].append(value)

    def get_data(self, key):
        return self.cache.get(key)

    def print_data(self, target):
        if target == 'smiles':
            print(f"Data type: SMILES; Data: {self.cache['SMILES']}")
        elif target == 'cid':
            print(f"Data type: SMILES; Data: {self.cache['CID']}")
        elif target == 'inchi':
            print(f"Data type: SMILES; Data: {self.cache['InChi']}")
        elif target == 'inchikey':
            print(f"Data type: SMILES; Data: {self.cache['InChiKey']}")
        elif target == 'name':
            print(f"Data type: SMILES; Data: {self.cache['Name']}")
        elif target == 'all':
            for key, value in self.cache.items():
                print(f"Data type: {key}; Data: {value}")

In [86]:
class FileManager:
    def __init__(self, directory):
        self.directory = directory

        # Create the directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)

    def save_file(self, identifier, file_content, file_extension):
        """Save a file with the given identifier and content"""
        file_path = os.path.join(self.directory, f"{identifier}.{file_extension}")
        with open(file_path, "w") as file:
            file.write(file_content)

    def load_file(self, identifier, file_extension):
        """Load a file with the given identifier"""
        file_path = os.path.join(self.directory, f"{identifier}.{file_extension}")
        if os.path.exists(file_path):
            with open(file_path, "r") as file:
                return file.read()
        else:
            return None

    def delete_file(self, identifier, file_extension):
        """Delete a file with the given identifier"""
        file_path = os.path.join(self.directory, f"{identifier}.{file_extension}")
        if os.path.exists(file_path):
            os.remove(file_path)

In [16]:
m1 = converter('SBASXUCJHJRPEV-UHFFFAOYSA-N', 'inchikey', cache)
m = m1.convert('smiles')
mn = m1.convert('name')
mi = m1.convert('inchi')

In [29]:
cache.print_data('all')

Data type: InChiKey; Data: ['SBASXUCJHJRPEV-UHFFFAOYSA-N']
Data type: SMILES; Data: ['COCCOCCO']
Data type: CID; Data: [8134]
Data type: Name; Data: ['2-(2-methoxyethoxy)ethanol']
Data type: InChi; Data: ['InChI=1S/C5H12O3/c1-7-4-5-8-3-2-6/h6H,2-5H2,1H3\n']


In [ ]:
def is_valid_molecule(molecule):
    smiles = get_smiles_from_name(molecule_name) if not Chem.MolFromSmiles(molecule_name) else molecule_name
    molecule = Chem.MolFromSmiles(smiles)
    return molecule is not None

view.setStyle({'stick': 'stick', 'line': 'line',{'linewidth': ''}, 'sphere': 'sphere'}[style])

In [13]:
style_dropdown = Dropdown(
    options=['line', 'stick', 'sphere','All'],
    value='All',
    description='Style:')

linewidth_slider = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    description='Line Width')

radius_slider = FloatSlider(
    value=0.2,
    min=0,
    max=1,
    step=0.1,
    description='Atomic radius size') 

scale_slider = FloatSlider(
    value=1,
    min=0,
    max=1,
    step=0.1,
    description='Sphere size')

In [14]:
def update_visibility(style):
    if style == 'line':
        linewidth_slider.layout.visibility = 'visible'
        radius_slider.layout.visibility = 'hidden'
        scale_slider.layout.visibility = 'hidden'
    elif style =='stick':
        linewidth_slider.layout.visibility = 'hidden'
        radius_slider.layout.visibility = 'visible'
        scale_slider.layout.visibility = 'hidden'
    elif style =='sphere':
        linewidth_slider.layout.visibility = 'hidden'
        radius_slider.layout.visibility = 'hidden'
        scale_slider.layout.visibility = 'visible'
    elif style == 'All':
        linewidth_slider.layout.visibility = 'visible'
        radius_slider.layout.visibility = 'visible'
        scale_slider.layout.visibility = 'visible'

In [31]:
def view3D(data_cache, identifier, style='All', linewidth='1', radius='0.2', scale='1'):
    """Visualize a molecule in 3D"""
    smiles_data = data_cache.get_data(identifier)
    print(smiles_data)
    if not smiles_data:
        print("SMILES not found in data cache")

    if 'SMILES' in smiles_data:
        smiles = smiles_data['SMILES']
    else:
        print("SMILES not found in data cache")
        return None

    if not smiles:
        print("Empty SMILES data")
        return None

    directory = './3Dfiles/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the file path for the SDF file
    sdf_file = os.path.join(directory, f"{identifier}.sdf")

    # Check if the SDF file already exists
    if not os.path.exists(sdf_file):
        # If not, convert SMILES to SDF and save it
        molecule = converter(smiles, 'smiles')
        sdf = molecule.convert('sdf')
        if not sdf:
            print("Failed to convert SMILES to SDF.")
            return None
        
        with open(sdf_file, 'w') as f:
            f.write(sdf)
    else:
        # If the SDF file exists, read it
        with open(sdf_file, 'r') as f:
            sdf = f.read()
        
    view = py3Dmol.view()
    view.addModel(open(name_file, 'r').read(),'sdf')
    view.setBackgroundColor('#000000', viewer=(0,0))
    
    if style == 'line':
        view.setStyle({'line': {'linewidth': linewidth}})
    elif style == 'stick':
        view.setStyle({'stick': {'radius': radius}})
    elif style == 'sphere':
        view.setStyle({'sphere': {'scale': scale}})
    elif style == 'All':
        view = py3Dmol.view(width=1500, height=400, viewergrid=(1,3), linked=True)
        view.addModel(open(name_file, 'r').read(),'sdf')
        view.setBackgroundColor('#000000', viewer=(0,0))
        view.setBackgroundColor('#000000', viewer=(0,1))
        view.setBackgroundColor('#000000', viewer=(0,2))
        view.setStyle({'line': {'linewidth': linewidth}}, viewer=(0,0))
        view.setStyle({'stick':{'radius': radius}}, viewer=(0,1))
        view.setStyle({'sphere': {'scale': scale}}, viewer=(0,2))
    return view

interact(view3D, data_cache=fixed(cache), identifier='COCCOCCO',
         style=style_dropdown,
         linewidth=linewidth_slider,
         radius=radius_slider,
         scale=scale_slider)

style_dropdown.observe(lambda change: update_visibility(change['new']), names='value')

interactive(children=(Text(value='COCCOCCO', description='identifier'), Dropdown(description='Style:', index=3…

In [187]:
m = converter('Erythronolide B', 'name')
name = m.convert('name')
cid = m.convert('cid')
smi = m.convert('smiles')
print(smi)
#name_file = name + '.sdf'
#pcp.download('SDF', name_file, cid, 'cid', overwrite=True)

CC[C@@H]1[C@@H]([C@@H]([C@H](C(=O)[C@@H](C[C@@]([C@@H]([C@H]([C@@H]([C@H](C(=O)O1)C)O)C)O)(C)O)C)C)O)C


In [24]:
indexs_checkbox = Checkbox(
    value = False,
    description = 'Index atoms (small)'
)

indexl_checkbox = Checkbox(
    value = False,
    description = 'Index atoms (large)'
)

stereo_checkbox = Checkbox(
    value = False,
    description = 'Show stereochemistry' 
)

substructure_input = Text(
    value='',
    description='Enter SMILES substructure'
)

abbr_checkbox = Checkbox(
    value = False,
    description = 'Use abbreviations'
)

In [25]:
def indexs_atoms(molecule):
    IPythonConsole.drawOptions.addAtomIndices = True
    return molecule

def indexl_atoms(molecule):
    for atom in molecule.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return molecule

def stereo_atoms(molecule):
    IPythonConsole.drawOptions.addStereoAnnotation = True
    rdCIPLabeler.AssignCIPLabels(molecule)
    return molecule

def get_substructure(molecule, smarts):
    substructure = Chem.MolFromSmarts(smarts)
    return molecule.GetSubstructMatches(substructure)

def vis_abbr(molecule):
    abbrevs = rdAbbreviations.GetDefaultAbbreviations()
    return rdAbbreviations.CondenseMolAbbreviations(molecule, abbrevs)

In [26]:
#CC(=O)OC1=CC=CC=C1C(=O)O
def view2D(smi, indexs = False, indexl = False, stereo = False, abbreviations =  False, substructure=''):
    molecule = Chem.MolFromSmiles(smi)    
    if molecule:
        if indexs:
            molecule = indexs_atoms(molecule)
        elif indexs == False:
            IPythonConsole.drawOptions.addAtomIndices = False
        if indexl:
            molecule = indexl_atoms(molecule)
        if stereo:
            molecule = stereo_atoms(molecule)
        elif stereo == False:
            IPythonConsole.drawOptions.addStereoAnnotation = False
        if abbreviations:
            molecule = vis_abbr(molecule)
        if substructure:
            matches = get_substructure(molecule, substructure)
        return molecule
    else:
        print(f"The molecule name '{molecule_name}' is not valid.")
        return None

def handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure):
    return view2D(smi, indexs, indexl, stereo, abbreviations, substructure)

interact(handle_checkbox, smi='', 
         indexs=indexs_checkbox, 
         indexl=indexl_checkbox, 
         stereo=stereo_checkbox,
         abbreviations=abbr_checkbox,
         substructure=substructure_input)

interactive(children=(Text(value='', description='smi'), Checkbox(value=False, description='Index atoms (small…

<function __main__.handle_checkbox(smi, indexs, indexl, stereo, abbreviations, substructure)>

In [ ]:
        if isinstance(self, (float,int)):
            raise ValueError(  
                f"Invalid input: '{self}'."
                " Input valid SMILES"
            )

        try:
            smi = str(self)
        
        except ValueError:     
            raise ValueError(  
                f"Invalid input: '{self}'."
                " `SMILES` must be a string, or convertible to a string."
                f" Original error message: {e}"
            )

        except IndexError:
            return None